# Twitter Data Updates

This Python code will be used to update and append the data sets created in `TwitterData-Start.ipynb` since we can only access the Twitter data of the last 7 days.

In [46]:
# Import Dependencies
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import date
from datetime import timedelta
import tweepy

from config import consumer_key
from config import consumer_secret
from config import access_token
from config import access_token_secret
from config import token

In [47]:
# API Keys
consumer_key = consumer_key
consumer_secret = consumer_secret
access_token = access_token
access_token_secret = access_token_secret
token = token

## Count of Gas & Oil Tweets

In [48]:
query = "gas oil -is:retweet"
client = tweepy.Client(bearer_token=token)
counts = client.get_recent_tweets_count(query=query, 
                                        granularity='day')

In [49]:
Lst = []
# Dict = {}
for i in counts.data:
    Dict = {}
    Dict['Day'] = str(i["start"][0:10])
    Dict['Tweet Count'] = i["tweet_count"]
    Lst.append(Dict)

In [50]:
df = pd.DataFrame(Lst)
df = pd.DataFrame.from_dict(Lst)
df = pd.DataFrame.from_records(Lst)
df = df.rename(columns={"Day": "Date"})
df['Date'] = pd.to_datetime(df['Date'])
# df.to_csv('tweet_count.csv', index=False) 
update_df = df.iloc[-2:-1]

In [51]:
update_df

,Date,Tweet Count
6,2022-12-17,3743


In [52]:
df

,Date,Tweet Count
0,2022-12-11,3561
1,2022-12-12,6529
2,2022-12-13,5490
3,2022-12-14,6096
4,2022-12-15,5283
5,2022-12-16,4970
6,2022-12-17,3743
7,2022-12-18,184


In [53]:
tweet_count = pd.read_csv("./tweet_count.csv")

In [54]:
tweet_count = tweet_count.rename(columns={"d": "Date"})
tweet_count['Date'] = pd.to_datetime(tweet_count['Date'])
tweet_count = pd.concat([tweet_count, update_df], ignore_index = True)

In [55]:
tweet_count = tweet_count.drop(columns=['Unnamed: 0'])
tweet_count.to_csv('tweet_count.csv')
tweet_count

,Date,Tweet Count
0,2022-11-23,5665
1,2022-11-24,4852
2,2022-11-25,4282
3,2022-11-26,3856
4,2022-11-27,3898
5,2022-11-28,5089
6,2022-11-29,6413
7,2022-11-30,7249
8,2022-12-01,5941
9,2022-12-02,5273


In [56]:
tweet_count.index = tweet_count.index.map(str)
tweet_count_dict = tweet_count.to_dict()
tweet_count_dict

{'Date': {'0': Timestamp('2022-11-23 00:00:00'),
  '1': Timestamp('2022-11-24 00:00:00'),
  '2': Timestamp('2022-11-25 00:00:00'),
  '3': Timestamp('2022-11-26 00:00:00'),
  '4': Timestamp('2022-11-27 00:00:00'),
  '5': Timestamp('2022-11-28 00:00:00'),
  '6': Timestamp('2022-11-29 00:00:00'),
  '7': Timestamp('2022-11-30 00:00:00'),
  '8': Timestamp('2022-12-01 00:00:00'),
  '9': Timestamp('2022-12-02 00:00:00'),
  '10': Timestamp('2022-12-03 00:00:00'),
  '11': Timestamp('2022-12-04 00:00:00'),
  '12': Timestamp('2022-12-05 00:00:00'),
  '13': Timestamp('2022-12-06 00:00:00'),
  '14': Timestamp('2022-12-07 00:00:00'),
  '15': Timestamp('2022-12-08 00:00:00'),
  '16': Timestamp('2022-12-09 00:00:00'),
  '17': Timestamp('2022-12-10 00:00:00'),
  '18': Timestamp('2022-12-11 00:00:00'),
  '19': Timestamp('2022-12-12 00:00:00'),
  '20': Timestamp('2022-12-13 00:00:00'),
  '21': Timestamp('2022-12-14 00:00:00'),
  '22': Timestamp('2022-12-15 00:00:00'),
  '23': Timestamp('2022-12-16 00:00:

## Adding Recent Tweets

In [57]:
# Returns the current local date
day1 = date.today()
day2 = day1 + timedelta(days = 1)
day1 = str(day1)
day2 = str(day2)

In [58]:
print(day1)
print(day2)

2022-12-17
2022-12-18


In [59]:
start = day1 + 'T00:00:00.000Z'
end = day2 + 'T00:00:00.00Z'

tweets_list = []

tweets = tweepy.Paginator(client.search_recent_tweets, query=query,
                          tweet_fields=['context_annotations', 'created_at'],
                          start_time=start, 
                          end_time=end,
                          max_results=100).flatten(limit=10000)

for tweet in tweets:
    tweets_list.append(tweet.text)
    
len(tweets_list)

3739

In [60]:
tweet_df = pd.DataFrame(tweets_list)
tweet_df.columns = [day1]
tweet_df = tweet_df.drop_duplicates()
tweet_df.to_csv(day1 + ".csv", index=True) 
len(tweet_df)

3675

In [61]:
all_tweets = pd.read_csv("./tweets.csv")

In [62]:
all_tweets = all_tweets.join(tweet_df, how='outer')

In [63]:
all_tweets = all_tweets.drop(columns=['Unnamed: 0'])
all_tweets

,2022-11-24,2022-11-25,2022-11-26,2022-11-27,2022-11-28,2022-11-29,2022-11-30,2022-12-01,2022-12-02,2022-12-03,...,2022-12-08,2022-12-09,2022-12-10,2022-12-11,2022-12-12,2022-12-13,2022-12-14,2022-12-15,2022-12-16,2022-12-17
0,@BellaWallerstei They have been paying oil and...,@TrainerGonzo @Belinda71555585 It's that easy ...,@HouseGOP They tried to stick the gas price hi...,@GilHough @KamalaHarris https://t.co/LBgWDBW9xs,"@ChrisMartzWX For well over 40 years, I have p...",@POTUS Except that gas prices were on the rise...,@FinancialReview simply nonsense. Don't let th...,@BBCNews You have a responsibility to explain ...,@GustavoBolivar Es una afrenta para la industr...,@SenateGOP Except the oil and gas companies do...,...,"Significant losses in Lumber, Coal &amp; Oil D...","Petro Yuan coming... \n\n""President Xi Jinping...",@JustinTrudeau Get rid of the carbon tax you m...,Isn’t it wonderful? Love to have inflation dow...,"Funding for new oil, gas and coal is fuelling ...",@Opioraryap @js7891m @elonmusk @TomFitton The ...,@SenCapito One: Inflation is worldwide and the...,@YALiberty @RepublicanChick @RandPaul https://...,@TheDemocrats U.S. in a major recession we hav...,Don Obadiah Marina Steinbeck #世界杯 Bridget S...
1,@BladeoftheS 3 The major difference between UK...,@davidmweissman Clueless! Right he inherited e...,@AnnieForTruth Yep. I remember when an attenda...,@NikkiHaley I agree use our own oil and coal a...,@WhiteHouse His executive orders on the energy...,@robreiner You really need help. Why are you s...,"@GavinNewsom You could end the gas tax, but yo...",@Marmee33 https://t.co/MeXmoIOgc9,@JDCocchiarella Jack; I can answer that questi...,@GOP And Biden has created more jobs in 2 year...,...,"@strategywoman Absolutely, it's essential to d...",@JDCocchiarella cause he caused the prices to ...,Big #Oil stocks are pointing to higher #gas pr...,@WhiteHouse You are LYING !!!!\nhttps://t.co/T...,"@VP Driving gas prices up to record highs, mak...",Joe Biden? Really? Is he the owner of gas/oil ...,@BhagatCPSU @IuliiaMendel Besides Finland stop...,"@RRapier In your Forbes article ""How Has The O...",@POTUS U.S. in a major recession we have no bo...,@KateGoesTech @SpiritofHo @McFaul China has te...
2,¡Oil &amp; Gas Reporter está disponible! https...,@laurenboebert Magic money? You mean from the ...,"@RBReich -\n...and the NATO Supremacists said,...",Nothing is being offered near Utah’s oil and g...,@manuel_rdgn @SEGOB_mx @adan_augusto @calderon...,TCS is a proud sponsor of the Oil &amp; Gatsby...,@POTUS Gas prices are down only because you ar...,@Sadiksmart7 @mohamedamiin2 @Qatari You can't ...,@diegowainstein El colectivo trucho de feminis...,"@SandyHasCandy @TomTSEC And somehow, in your b...",...,¡Oil &amp; Gas Reporter está disponible! https...,“Xi Jinping vowed on Friday to import more oil...,@POTUS We sure as hell aren't!! It's dark days...,"@nbc_brian @GregMcLeanYYC @cafreeland Brain, G...",@FedUp9876 @FoxNews I've been in the financial...,@lukemulks @PhillyD What are the main policies...,@fuckcancer56 @SenJohnKennedy If that's the ca...,@WhiteHouse My heating oil is double what I pa...,@WhiteHouse U.S. in a major recession we have ...,Training Your Dog to Detect Leaks In Pipelines...
3,@binnahar85 Europe wants to boycott Qatar? Are...,@thetruthsucks12 @haliwell51 @gmbutts No maybe...,@Princemoye1 I thought its about those that ki...,@Scott19490 @isaiahrmartin Yeah because relian...,rtr: U.S. seeks to limit flaring and methane l...,Should California tax oil profits? Gas spike h...,https://t.co/HEMuIIrFin Oil “Quality News Link...,¡Oil &amp; Gas Reporter está disponible! https...,"Cut off from Russian gas and oil imports, much...",@MichelleSteelCA If only our idiot Prime Minis...,...,@MattWeaverRA Crude oil can be refined into mo...,The #Palantir Foundry Ontology is what you tho...,"@impetro9 I mean, just to play devils advocate...",¡Oil &amp; Gas Reporter está disponible! https...,@AC_NoChill Oil and gas comes and goes. Digi

In [64]:
all_tweets.to_csv('tweets.csv')

In [65]:
all_tweets = all_tweets.dropna()
all_tweets.to_csv('tweets_clean.csv')

In [66]:
all_tweets

,2022-11-24,2022-11-25,2022-11-26,2022-11-27,2022-11-28,2022-11-29,2022-11-30,2022-12-01,2022-12-02,2022-12-03,...,2022-12-08,2022-12-09,2022-12-10,2022-12-11,2022-12-12,2022-12-13,2022-12-14,2022-12-15,2022-12-16,2022-12-17
0,@BellaWallerstei They have been paying oil and...,@TrainerGonzo @Belinda71555585 It's that easy ...,@HouseGOP They tried to stick the gas price hi...,@GilHough @KamalaHarris https://t.co/LBgWDBW9xs,"@ChrisMartzWX For well over 40 years, I have p...",@POTUS Except that gas prices were on the rise...,@FinancialReview simply nonsense. Don't let th...,@BBCNews You have a responsibility to explain ...,@GustavoBolivar Es una afrenta para la industr...,@SenateGOP Except the oil and gas companies do...,...,"Significant losses in Lumber, Coal &amp; Oil D...","Petro Yuan coming... \n\n""President Xi Jinping...",@JustinTrudeau Get rid of the carbon tax you m...,Isn’t it wonderful? Love to have inflation dow...,"Funding for new oil, gas and coal is fuelling ...",@Opioraryap @js7891m @elonmusk @TomFitton The ...,@SenCapito One: Inflation is worldwide and the...,@YALiberty @RepublicanChick @RandPaul https://...,@TheDemocrats U.S. in a major recession we hav...,Don Obadiah Marina Steinbeck #世界杯 Bridget S...
1,@BladeoftheS 3 The major difference between UK...,@davidmweissman Clueless! Right he inherited e...,@AnnieForTruth Yep. I remember when an attenda...,@NikkiHaley I agree use our own oil and coal a...,@WhiteHouse His executive orders on the energy...,@robreiner You really need help. Why are you s...,"@GavinNewsom You could end the gas tax, but yo...",@Marmee33 https://t.co/MeXmoIOgc9,@JDCocchiarella Jack; I can answer that questi...,@GOP And Biden has created more jobs in 2 year...,...,"@strategywoman Absolutely, it's essential to d...",@JDCocchiarella cause he caused the prices to ...,Big #Oil stocks are pointing to higher #gas pr...,@WhiteHouse You are LYING !!!!\nhttps://t.co/T...,"@VP Driving gas prices up to record highs, mak...",Joe Biden? Really? Is he the owner of gas/oil ...,@BhagatCPSU @IuliiaMendel Besides Finland stop...,"@RRapier In your Forbes article ""How Has The O...",@POTUS U.S. in a major recession we have no bo...,@KateGoesTech @SpiritofHo @McFaul China has te...
2,¡Oil &amp; Gas Reporter está disponible! https...,@laurenboebert Magic money? You mean from the ...,"@RBReich -\n...and the NATO Supremacists said,...",Nothing is being offered near Utah’s oil and g...,@manuel_rdgn @SEGOB_mx @adan_augusto @calderon...,TCS is a proud sponsor of the Oil &amp; Gatsby...,@POTUS Gas prices are down only because you ar...,@Sadiksmart7 @mohamedamiin2 @Qatari You can't ...,@diegowainstein El colectivo trucho de feminis...,"@SandyHasCandy @TomTSEC And somehow, in your b...",...,¡Oil &amp; Gas Reporter está disponible! https...,“Xi Jinping vowed on Friday to import more oil...,@POTUS We sure as hell aren't!! It's dark days...,"@nbc_brian @GregMcLeanYYC @cafreeland Brain, G...",@FedUp9876 @FoxNews I've been in the financial...,@lukemulks @PhillyD What are the main policies...,@fuckcancer56 @SenJohnKennedy If that's the ca...,@WhiteHouse My heating oil is double what I pa...,@WhiteHouse U.S. in a major recession we have ...,Training Your Dog to Detect Leaks In Pipelines...
4,@TheoFleury14 This was always about $$ and the...,You are full of it. You shut down our clean cr...,The big story. Proof that Westminster has been...,@RepBoebert Lauren Boebert’s income is almost ...,¡Oil &amp; Gas Reporter está disponible! https...,This is a classic example of Washington corrup...,@SenatorLankford You do know that US is the la...,#Gold #Silver and #Crypto update for 12/01/22 ...,¡Oil &amp; Gas Reporter está disponible! https...,PUMP AMERICAN OIL/NG NOW TO SAVE THE US ECONOM...,...,Ottawa unveils policy to end public funding fo...,#ICYMI Ottawa moves to end subsidies for new o...,¡Oil &amp; Gas Reporter está disponible! https...,@WhiteHouse Yes starve us of energy nothing lo...,Our public lands are at risk. Despite @POTUS's..

In [67]:
all_tweets.index = all_tweets.index.map(str)
all_tweets_dict = all_tweets.to_dict()

In [168]:
ts_df = pd.DataFrame()
ts_list = []
append_data = []

for (columnName, columnData) in all_tweets.iteritems():
    case = {columnName : columnData.values}
    ts_list.append(case)

for i in range(len(ts_list)):
    ts_series = pd.Series(ts_list[i], name='Raw_Text').rename_axis('Date').explode().reset_index()
    append_data.append(ts_series)
ts_df = pd.concat(append_data)
ts_df = ts_df.reset_index()
ts_df = ts_df.drop(columns=['index'])

ts_df.to_csv('transposed_tweets.csv')

ts_df.index = ts_df.index.map(str)
transposed_tweets_dict = ts_df.to_dict()